In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from sklearn.kernel_ridge import KernelRidge
from sklearn.metrics import mean_absolute_error, mean_squared_error
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/tabular-playground-series-jun-2022/sample_submission.csv
/kaggle/input/tabular-playground-series-jun-2022/data.csv


In [2]:
# Kaggle given data url is set to this string
data_url = '/kaggle/input/tabular-playground-series-jun-2022/data.csv'
# Reading the un-edited raw data as csv
raw_data = pd.read_csv(data_url, index_col='row_id')

In [3]:
def reduce_mem_usage(train_data):
    """ iterate through all the columns of a dataframe and modify the data type
    to reduce memory usage.
    """
    start_mem = train_data.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))

    for col in train_data.columns:
        col_type = train_data[col].dtype

        if col_type != object:
            c_min = train_data[col].min()
            c_max = train_data[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    train_data[col] = train_data[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    train_data[col] = train_data[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    train_data[col] = train_data[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    train_data[col] = train_data[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    train_data[col] = train_data[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    train_data[col] = train_data[col].astype(np.float32)
                else:
                    train_data[col] = train_data[col].astype(np.float64)
        else:
            train_data[col] = train_data[col].astype('category')

    end_mem = train_data.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))

    return train_data

In [4]:
compressed_data = reduce_mem_usage(raw_data)
del raw_data

Memory usage of dataframe is 617.98 MB
Memory usage after optimization is: 136.38 MB
Decreased by 77.9%


In [4]:
display(raw_data.head(3))
display(raw_data.info())
print('\nData size if all NaN containing rows are dropped :', raw_data.dropna().shape[0])

,F_1_0,F_1_1,F_1_2,F_1_3,F_1_4,F_1_5,F_1_6,F_1_7,F_1_8,F_1_9,...,F_4_5,F_4_6,F_4_7,F_4_8,F_4_9,F_4_10,F_4_11,F_4_12,F_4_13,F_4_14
row_id,,,,,,,,,,,,,,,,,,,,,
0,-0.354591,-0.464038,2.304115,0.734486,1.696395,0.136285,-0.518344,0.502640,-1.852504,-0.500665,...,3.744152,0.794438,0.265185,-0.561809,0.196480,0.373434,6.206995,3.809505,1.236486,1.182055
1,1.380940,-0.499626,-0.418548,1.911725,-0.826130,-1.715371,-0.577091,-1.041486,0.596067,-0.363425,...,-2.895826,-0.738275,2.361818,-0.060753,0.727249,-0.271882,5.232157,-4.218259,-2.724883,-0.063775
2,0.256023,-1.059874,NaN,0.345678,1.513814,1.243864,-0.509648,-0.800481,-0.115945,0.595777,...,2.252834,0.472496,2.491386,0.353381,-0.260682,-0.000833,-0.116457,-2.131747,3.661499,-0.131576


<class 'pandas.core.frame.DataFrame'>
Int64Index: 1000000 entries, 0 to 999999
Data columns (total 80 columns):
 #   Column  Non-Null Count    Dtype  
---  ------  --------------    -----  
 0   F_1_0   981603 non-null   float64
 1   F_1_1   981784 non-null   float64
 2   F_1_2   981992 non-null   float64
 3   F_1_3   981750 non-null   float64
 4   F_1_4   981678 non-null   float64
 5   F_1_5   981911 non-null   float64
 6   F_1_6   981867 non-null   float64
 7   F_1_7   981872 non-null   float64
 8   F_1_8   981838 non-null   float64
 9   F_1_9   981751 non-null   float64
 10  F_1_10  982039 non-null   float64
 11  F_1_11  981830 non-null   float64
 12  F_1_12  981797 non-null   float64
 13  F_1_13  981602 non-null   float64
 14  F_1_14  981961 non-null   float64
 15  F_2_0   1000000 non-null  int64  
 16  F_2_1   1000000 non-null  int64  
 17  F_2_2   1000000 non-null  int64  
 18  F_2_3   1000000 non-null  int64  
 19  F_2_4   1000000 non-null  int64  
 20  F_2_5   1000000 non-null 

None


Data size if all NaN containing rows are dropped : 364774


In [5]:
for raw_number in range(1,5):
    temp_column_name = "F_"+str(raw_number)
    sub_columns = [c for c in raw_data.columns if temp_column_name in c]
    display(raw_data[sub_columns].describe().transpose())

,count,mean,std,min,25%,50%,75%,max
F_1_0,981603.0,-0.000687,1.000784,-4.664447,-0.674667,-0.000769,0.673181,5.039177
F_1_1,981784.0,0.002091,0.999878,-4.790594,-0.672284,0.002049,0.675997,5.042807
F_1_2,981992.0,0.000551,0.999825,-4.870645,-0.673783,0.001386,0.674490,5.130025
F_1_3,981750.0,0.000982,0.999847,-5.053023,-0.672211,0.000370,0.674852,5.462472
F_1_4,981678.0,0.002437,1.001151,-5.363454,-0.672326,0.002733,0.676588,4.856916
F_1_5,981911.0,0.000635,0.999927,-5.508223,-0.673884,0.000276,0.675951,4.960724
F_1_6,981867.0,-0.000124,1.000427,-5.199170,-0.674834,0.000814,0.673983,4.957977
F_1_7,981872.0,-0.063857,0.725986,-6.990302,-0.500305,0.000578,0.444337,2.527570
F_1_8,981838.0,-0.000014,1.000715,-4.566593,-0.674006,-0.000047,0.674295,4.886075
F_1_9,981751.0,0.000451,1.000194,-4.997549,-0.673513,0.001118,0.675575,4.788579


,count,mean,std,min,25%,50%,75%,max
F_2_0,1000000.0,2.687703,1.875394,0.0,1.0,2.0,4.0,15.0
F_2_1,1000000.0,2.514404,1.751238,0.0,1.0,2.0,4.0,14.0
F_2_2,1000000.0,0.976758,1.038289,0.0,0.0,1.0,2.0,11.0
F_2_3,1000000.0,2.517147,1.653054,0.0,1.0,2.0,4.0,14.0
F_2_4,1000000.0,2.941194,1.976869,0.0,1.0,3.0,4.0,16.0
F_2_5,1000000.0,1.532953,1.351220,0.0,1.0,1.0,2.0,12.0
F_2_6,1000000.0,1.491746,1.315535,0.0,0.0,1.0,2.0,12.0
F_2_7,1000000.0,2.645875,1.744770,0.0,1.0,2.0,4.0,16.0
F_2_8,1000000.0,1.177792,1.318677,0.0,0.0,1.0,2.0,13.0
F_2_9,1000000.0,1.111291,1.095853,0.0,0.0,1.0,2.0,11.0


,count,mean,std,min,25%,50%,75%,max
F_3_0,981971.0,0.001737,1.001323,-4.693600,-0.674649,0.003252,0.676849,4.586671
F_3_1,981655.0,-0.001147,1.000467,-4.466075,-0.674786,0.000481,0.673515,4.851365
F_3_2,981944.0,0.000605,0.998939,-4.885859,-0.673464,0.000392,0.674722,4.763391
F_3_3,981946.0,0.000834,1.000365,-4.677193,-0.673968,0.000855,0.675631,4.988126
F_3_4,981627.0,0.001290,1.000016,-5.009163,-0.672924,0.002639,0.676753,4.721604
F_3_5,981702.0,-0.002180,1.000120,-4.870645,-0.676311,-0.001597,0.671564,5.039177
F_3_6,981808.0,0.000058,0.999345,-5.018589,-0.675043,0.000855,0.673441,4.525290
F_3_7,981987.0,0.001525,1.000698,-5.053023,-0.673119,0.001199,0.675634,5.462472
F_3_8,981902.0,0.000773,1.001004,-5.508223,-0.675624,-0.000019,0.675939,5.106927
F_3_9,981894.0,-0.000440,1.001646,-4.846381,-0.675138,-0.001767,0.674269,5.101094


,count,mean,std,min,25%,50%,75%,max
F_4_0,981872.0,0.326608,2.321696,-12.878366,-1.167786,0.421343,1.908990,10.656593
F_4_1,981836.0,-0.330865,2.406335,-12.532302,-1.962011,-0.356394,1.276280,11.673864
F_4_2,981505.0,-0.085792,0.837157,-9.663044,-0.608231,-0.061980,0.485107,2.908549
F_4_3,981971.0,-0.195488,0.820821,-9.942220,-0.685531,-0.137238,0.368864,2.581990
F_4_4,982043.0,0.333060,2.373767,-12.825041,-1.190535,0.424968,1.943201,11.927110
F_4_5,981937.0,0.335968,2.352749,-12.536923,-1.266885,0.303320,1.917695,13.539926
F_4_6,981675.0,0.003773,2.292272,-11.132350,-1.570766,-0.071800,1.521134,11.524512
F_4_7,981986.0,0.334433,2.358757,-11.677862,-1.224468,0.379014,1.925968,12.536151
F_4_8,981824.0,-0.071842,0.777855,-10.091950,-0.517559,0.018222,0.474558,2.607285
F_4_9,981735.0,-0.079854,0.806965,-9.864310,-0.576799,-0.027811,0.480100,2.814969


In [10]:
def kernel_liner_reg(train_X, train_y, validation_X, validation_y, test_X, test_y):
    krr = KernelRidge(alpha=1.0)
    krr.fit(train_X, train_y)
    validation_prediction = krr.predict(validation_X)
    test_prediction = krr.predict(test_X)
    mae_val = mean_absolute_error(validation_y, validation_prediction)
    mse_val = mean_squared_error(validation_y, validation_prediction)
    return krr, mae_val, mse_val, test_prediction

In [11]:
def train_model(data):
    # Iterating through the columns
    for column in data.columns:
        if 'F_2' not in column:
            column_list = data.columns.values.tolist() # Column names as list
            column_list.remove(column) # Column names list without the column being iterated
            # row_id numbers where iterated column is NaN
            nan_index = data[column].index[data[column].apply(np.isnan)]
            # row_id number where iterated column is not NaN
            not_nan_index = data[column].index[data[column].apply(pd.notnull)]
            # Get rows where iterated column value is NaN
            temp_test_data = data.iloc[nan_index]
            # Get rows where iterated column value is not NaN
            temp_train_data = data.iloc[not_nan_index]
            # Fill the other column NaN values with their mean
            filled_test_data = temp_test_data[column_list].fillna(data[column_list].mean())
            filled_test_data[column] = temp_test_data[column]
            del temp_test_data
            filled_train_data = temp_train_data[column_list].fillna(data[column_list].mean())
            filled_train_data[column] = temp_train_data[column]
            del temp_train_data
            # Acquire the test data
            test_data_X = filled_test_data[column_list]
            test_data_y = filled_test_data[column]
            del filled_test_data
            sampled_train_data = filled_train_data.sample(n=filled_train_data.shape[0],random_state=1)
            del filled_train_data
            train_size = int(sampled_train_data.shape[0]*(1-0.1))
            sampled_train_X = sampled_train_data[column_list]
            sampled_train_y = sampled_train_data[column]
            del sampled_train_data
            standard_X = (sampled_train_X - sampled_train_X.mean())/sampled_train_X.std()
            del sampled_train_X
            train_data_X = standard_X[:train_size]
            validation_data_X = standard_X[train_size:]
            train_data_y = sampled_train_y[:train_size]
            validation_data_y = sampled_train_y[train_size:]
            del sampled_train_y, standard_X
            _, mae_val, mse_val, test_pred = kernel_liner_reg(
                train_data_X, train_data_y,
                validation_data_X, validation_data_y,
                test_data_X, test_data_y
            )
            print('Column name:',column,
                  '\nValidation MAE:', mae_val,
                  '\nValidation MSE:', mse_val)

In [12]:
train_model(compressed_data)

/opt/conda/lib/python3.7/site-packages/numpy/core/_methods.py:48: RuntimeWarning: overflow encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims, initial, where)
/opt/conda/lib/python3.7/site-packages/numpy/core/fromnumeric.py:86: RuntimeWarning: invalid value encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [16]:
raw_data.std()

F_1_0     1.000784
F_1_1     0.999878
F_1_2     0.999825
F_1_3     0.999847
F_1_4     1.001151
            ...   
F_4_10    0.706913
F_4_11    4.996632
F_4_12    2.383485
F_4_13    2.364036
F_4_14    0.776354
Length: 80, dtype: float64